In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [2]:
#reading in sticker type data
feature_data = sqlContext.read.parquet("gs://ds-url-catag/stick_statistics/derived_features/")


In [ ]:
#convert pyspark to pandas dataframe
import pandas as pd
import pyarrow.parquet as pq
feature_data_df  = feature_data.toPandas()

In [ ]:
#checking size of dataframe
print 'Size of the dataframe: {}'.format(feature_data_df.shape)

In [ ]:
#logarithmic transformation of two different variables
import numpy as np
feature_data_df['log_noofdays']=np.log10(1+feature_data_df.numofdays)
feature_data_df['log_avg_con_days']=np.log10(1+feature_data_df.avg_con_days)
feature_data_df['log_sticker_packs_sent']=np.log10(1+feature_data_df.sticker_packs_sent)
feature_data_df['log_distinct_sticker_packs_sent']=np.log10(1+feature_data_df.distinct_sticker_packs_sent)
feature_data_df['log_sum_paid']=np.log10(1+feature_data_df.sum_paid)
feature_data_df['log_sum_free']=np.log10(1+feature_data_df.sum_free)
feature_data_df['log_sum_subs']=np.log10(1+feature_data_df.sum_subs)
feature_data_df['log_sum_discont']=np.log10(1+feature_data_df.sum_discont)


In [ ]:
import numpy as np
rndperm = np.random.permutation(feature_data_df.shape[0])

In [58]:
#applying tsne
import time

from sklearn.manifold import TSNE

n_sne = 7000

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature_data_df.loc[rndperm[:n_sne],['log_noofdays','log_avg_con_days','log_sticker_packs_sent','log_distinct_sticker_packs_sent','log_sum_paid','log_sum_free','log_sum_subs','log_sum_discont']].values)

print 't-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start)

Explained variation per principal component: [ 0.84617291  0.06662387  0.05912116]


In [2]:
from ggplot import *

df_tsne = tsne_results.loc[rndperm[:n_sne],:].copy()
df_tsne['x-tsne'] = tsne_results[:,0]
df_tsne['y-tsne'] = tsne_results[:,1]

chart = ggplot(df_tsne, aes(x='x-tsne', y='y-tsne',color = 'label') ) \
        + geom_point(size=70,alpha=0.1) \
        + ggtitle("tSNE dimensions colored by digit")
chart




NameError: name 'tsne_results' is not defined

In [105]:
# new_df = feature_data_df.as_matrix(columns=feature_data_df.columns[17:19])

In [ ]:
#clustering using Kmeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
model = kmeans.fit(feature_data_df[['pca-one','pca-two']])
y_kmeans = kmeans.predict(feature_data_df[['pca-one','pca-two']])

In [ ]:
centers = kmeans.cluster_centers_
plt.scatter(new_df[:, 0], new_df[:, 1], c=y_kmeans, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
plt.show()

In [ ]:
print new_df

In [138]:
transformed = model.transform(feature_data_df[['pca-one','pca-two'])

In [139]:
transformed.shape

(7495040, 4)

In [140]:
print transformed

[[ 1.24694752  0.66931198  1.23084474  0.46098885]
 [ 0.77320752  0.91870339  1.54907061  0.17817918]
 [ 1.93216153  0.41153628  0.53766549  1.02081789]
 ..., 
 [ 2.21084646  0.52211095  0.1293274   1.26070751]
 [ 1.92954715  0.23933699  0.39210293  0.97965731]
 [ 1.96963736  0.32512993  0.39279017  1.0311095 ]]


In [ ]:
transformed.groupby('')